In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

## config_entity.py

In [1]:
from dataclasses import dataclass
from finance_complaint.entity.config_entity import TrainingPipelineConfig
from finance_complaint.exception import FinanceException
from finance_complaint.logger import logging as logger
import os, sys

Reading environment variables
Read Complete!


In [2]:
MODEL_TRAINER_MODEL_NAME = "finance_estimator"

# Model Pusher Constants
MODEL_PUSHER_SAVED_MODEL_DIRS = "saved_models"
MODEL_PUSHER_DIR = "model_pusher"
MODEL_PUSHER_MODEL_NAME = MODEL_TRAINER_MODEL_NAME

In [6]:
@dataclass
class ModelPusherConfig:

    def __init__(self, training_pipeline_config: TrainingPipelineConfig) :
        try:
            self.pusher_model_dir = os.path.join(training_pipeline_config.artifact_dir, MODEL_PUSHER_DIR, "model", MODEL_PUSHER_MODEL_NAME)
            self.saved_model_dir = MODEL_PUSHER_SAVED_MODEL_DIRS
        except Exception as e:
            raise FinanceException(e, sys)

## artifact_entity.py

In [7]:
@dataclass
class ModelPusherArtifact:
    model_pushed_dir: str
    saved_model_dir: str

## model_pusher.py

In [5]:
import sys
import os

from finance_complaint.exception import FinanceException
from finance_complaint.logger import logging as  logger

# from finance_complaint.entity.config_entity import ModelPusherConfig
from finance_complaint.entity.artifact_entity import ModelTrainerArtifact#, ModelPusherArtifact

from finance_complaint.ml.estimator import ModelResolver

from pyspark.ml.pipeline import PipelineModel

In [8]:
class ModelPusher:

    def __init__(self, model_trainer_artifact: ModelTrainerArtifact, model_pusher_config: ModelPusherConfig):
        self.model_trainer_artifact = model_trainer_artifact
        self.model_pusher_config = model_pusher_config
        self.model_resolver = ModelResolver(model_dir=self.model_pusher_config.saved_model_dir)

    def push_model(self) -> str:
        try:
            trained_model_path=self.model_trainer_artifact.model_trainer_ref_artifact.trained_model_file_path
            saved_model_path = self.model_resolver.get_save_model_path
            model = PipelineModel.load(trained_model_path)
            model.save(saved_model_path)
            model.save(self.model_pusher_config.pusher_model_dir)
            return saved_model_path
        except Exception as e:
            raise FinanceException(e, sys)

    def initiate_model_pusher(self) -> ModelPusherArtifact:
        try:
            saved_model_path = self.push_model()
            model_pusher_artifact = ModelPusherArtifact(model_pushed_dir=self.model_pusher_config.pusher_model_dir,
                                    saved_model_dir=saved_model_path)
            logger.info(f"Model pusher artifact: {model_pusher_artifact}")
            return model_pusher_artifact
        except Exception as e:
            raise FinanceException(e, sys)
        


## training.py

In [11]:
from finance_complaint.exception import FinanceException
from finance_complaint.logger import logging as logger
from finance_complaint.entity.schema import FinanceDataSchema

from finance_complaint.entity.config_entity import (DataIngestionConfig, 
                                                    DataValidationConfig,
                                                    DataTransformationConfig,
                                                    ModelTrainerConfig,
                                                    ModelEvaluationConfig)
from finance_complaint.entity.artifact_entity import (DataIngestionArtifact,
                                                      DataValidationArtifact,
                                                      DataTransformationArtifact,
                                                      ModelTrainerArtifact,
                                                      ModelEvaluationArtifact)

from finance_complaint.components.data_ingestion import DataIngestion
from finance_complaint.components.data_validation import DataValidation
from finance_complaint.components.data_transformation import DataTransformation
from finance_complaint.components.model_trainer import ModelTrainer
from finance_complaint.components.model_evalutaion import ModelEvaluation

import sys, os


In [13]:
class TrainingPipeline:
    """ 
    this is a training pipeline comprising each components from flow chart
    """
    def __init__(self, training_pipeline_config: TrainingPipelineConfig):
        self.training_pipeline_config: TrainingPipelineConfig = training_pipeline_config

    def start_data_ingestion(self) -> DataIngestionArtifact:
        """
        responsible for starting data ingestion 
        """
        try:
            data_ingestion_config = DataIngestionConfig(training_pipeline_config=self.training_pipeline_config)
            data_ingestion = DataIngestion(data_ingestion_config=data_ingestion_config)
            data_ingestion_artifact = data_ingestion.initiate_data_ingestion()
            return data_ingestion_artifact
        
        except Exception as e:
            logger.debug(f"Error: {e}")
            raise FinanceException(e, sys)
        
    def start_data_validation(self, data_ingestion_artifact: DataIngestionArtifact) -> DataValidationArtifact:
        """
        takes ingested data artifact and validate that data by generating validation artifact
        """
        try:
            data_validation_config = DataValidationConfig(training_pipeline_config=self.training_pipeline_config)
            data_validation = DataValidation(
                data_validation_config=data_validation_config,
                data_ingestion_artifact=data_ingestion_artifact
            )
            data_validation_artifact = data_validation.initiate_data_validation()
            return data_validation_artifact
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start_data_transformation(self, data_validation_artifact: DataValidationArtifact) -> DataTransformationArtifact:
        """ 
        responsible for data transformation 
        """
        try:
            data_transformation_config = DataTransformationConfig(training_pipeline_config=self.training_pipeline_config)
            data_transformation = DataTransformation(data_validation_artifact=data_validation_artifact,
                                                     data_transformation_config=data_transformation_config)
            
            data_transformation_artifacts = data_transformation.initiate_data_transformation()
            return data_transformation_artifacts
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start_model_trainer(self, data_transformation_artifact: DataTransformationArtifact) -> ModelTrainerArtifact:
        try:
            model_trainer_config = ModelTrainerConfig(training_pipeline_config=self.training_pipeline_config)
            model_trainer = ModelTrainer(data_transformation_artifact=data_transformation_artifact,
                                         model_trainer_config=model_trainer_config)
            model_trainer_artifact = model_trainer.initiate_model_training()

            return model_trainer_artifact
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start_model_evaluation(self, data_validation_artifact, model_trainer_artifact) -> ModelEvaluationArtifact:
        try:
            model_eval_config = ModelEvaluationConfig(training_pipeline_config=self.training_pipeline_config)
            model_eval = ModelEvaluation(data_validation_artifact=data_validation_artifact,
                                        model_trainer_artifact=model_trainer_artifact,
                                        model_eval_config=model_eval_config
                                        )
            return model_eval.initiate_model_evaluation()
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start_model_pusher(self, model_trainer_artifact: ModelTrainerArtifact):
        try:


            model_pusher_config = ModelPusherConfig(training_pipeline_config=self.training_pipeline_config)
            model_pusher = ModelPusher(model_trainer_artifact=model_trainer_artifact,
                                       model_pusher_config=model_pusher_config
                                       )
            return model_pusher.initiate_model_pusher()
        except Exception as e:
            raise FinanceException(e, sys)


        
    def start(self):
        try:
            # initalizating data ingestion
            data_ingestion_artifact = self.start_data_ingestion()
            # initalizing data validation
            data_validation_artifact = self.start_data_validation(data_ingestion_artifact=data_ingestion_artifact)
            # initalizing data transformation
            data_transformation_artifact = self.start_data_transformation(data_validation_artifact=data_validation_artifact)
            # initalizing model training
            model_trainer_artifact = self.start_model_trainer(data_transformation_artifact=data_transformation_artifact)
            # initalizing model evaluation
            model_eval_artifact = self.start_model_evaluation(data_validation_artifact=data_validation_artifact,
                                                              model_trainer_artifact=model_trainer_artifact
                                                              )
            # initalizing model pusher
            if model_eval_artifact.model_accepted:
                self.start_model_pusher(model_trainer_artifact=model_trainer_artifact)

        except Exception as e:
            raise FinanceException(e, sys)    
        


## train.py

In [ ]:
from finance_complaint.pipeline.training import TrainingPipeline
# from finance_complaint.entity.config_entity import TrainingPipelineConfig

if __name__ == "__main__":
    training_pipeline_config = TrainingPipelineConfig()
    training_pipeline = TrainingPipeline(training_pipeline_config=training_pipeline_config)
    training_pipeline.start()